In [16]:
from scipy.io import loadmat
data = loadmat('trainingData.mat')
data = data['dataRecord']


In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
torch.backends.quantized.engine = 'qnnpack'
# 假设你的数据是numpy数组形式，需要转换成torch Tensor
import numpy as np

# 示例数据，你应该用你的实际数据替换这里的np.random.rand()
inputs = torch.tensor(data[:, :6], dtype=torch.float32)
outputs = torch.tensor(data[:, 6:], dtype=torch.float32)

# 创建数据集和数据加载器
dataset = TensorDataset(inputs, outputs)
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

# 定义神经网络模型
class SimpleFNN(nn.Module):
    def __init__(self):
        super(SimpleFNN, self).__init__()
        self.fc1 = nn.Linear(6, 32)  # 输入层到隐藏层，50个神经元
        self.fc2 = nn.Linear(32, 16) # 第二个隐藏层，20个神经元
        self.fc3 = nn.Linear(16, 3)  # 输出层

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

model = SimpleFNN()
model.train()

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练网络
epochs = 200
for epoch in range(epochs):
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')


Epoch 1, Loss: 0.0017105248989537358
Epoch 2, Loss: 9.046952072822023e-06
Epoch 3, Loss: 2.9752009140793234e-06
Epoch 4, Loss: 3.5471850878820987e-06
Epoch 5, Loss: 2.8625775030377554e-06
Epoch 6, Loss: 3.2645416467858013e-06
Epoch 7, Loss: 2.784619709927938e-06
Epoch 8, Loss: 1.5503923123105778e-06
Epoch 9, Loss: 3.5483005831338232e-06
Epoch 10, Loss: 1.4757727058167802e-06
Epoch 11, Loss: 2.882211902033305e-06
Epoch 12, Loss: 1.3056451280135661e-06
Epoch 13, Loss: 1.98003158402571e-06
Epoch 14, Loss: 2.088607061523362e-06
Epoch 15, Loss: 2.4746873350522947e-06
Epoch 16, Loss: 1.9437204628047766e-06
Epoch 17, Loss: 2.107668251483119e-06
Epoch 18, Loss: 1.845972974479082e-06
Epoch 19, Loss: 2.0836127987422515e-06
Epoch 20, Loss: 3.0048245207581203e-06
Epoch 21, Loss: 3.5596297038864577e-06
Epoch 22, Loss: 3.3967039598792326e-06
Epoch 23, Loss: 2.9544457902375143e-06
Epoch 24, Loss: 2.176181624236051e-06
Epoch 25, Loss: 1.972784048120957e-06
Epoch 26, Loss: 1.6268706986011239e-06
Epoch 

In [18]:
# infer and save
test_input = torch.tensor([[0.11412254, -0.09698317, -0.13648682, -0.08168529,  0.09584415,
        -0.08081015]], dtype=torch.float32)  # 示例输入数据
with torch.no_grad():
    prediction = model(test_input)
    print(f'Prediction: {prediction.numpy()}')
    print("-0.01443261, -0.01519005, -0.02687549")
# save the model
scripted_model = torch.jit.script(model)
torch.jit.save(scripted_model, '../model/model.pth')

Prediction: [[-0.01493122 -0.01529542 -0.02691081]]
-0.01443261, -0.01519005, -0.02687549


In [15]:
model.eval()  # 设置为评估模式
model.qconfig = torch.quantization.get_default_qconfig('qnnpack')
torch.quantization.prepare(model, inplace=True)

# 校准模型
with torch.no_grad():
    for _ in range(10):  # 使用部分数据进行校准
        model(inputs)

# 转换为量化模型
quantized_model = torch.quantization.convert(model, inplace=True)

# 保存为TorchScript
scripted_quant_model = torch.jit.script(quantized_model)
scripted_quant_model.save("quantized_model.pt")

In [6]:
data = loadmat('testingData.mat')
test_data = data['dataRecord']

# 测试数据集，应该与训练数据集具有相同的结构
test_inputs = torch.tensor(test_data[:, :6], dtype=torch.float32)
test_outputs = torch.tensor(test_data[:, 6:], dtype=torch.float32)

# 创建测试 DataLoader
test_dataset = TensorDataset(test_inputs, test_outputs)
test_loader = DataLoader(test_dataset, batch_size=10)

# 将模型设置为评估模式
model.eval()

# 初始化损失计算
test_loss = 0.0
with torch.no_grad():  # 确保不进行梯度计算
    for inputs, labels in test_loader:
        # 前向传播
        outputs = model(inputs)
        # 计算损失
        loss = criterion(outputs, labels)
        test_loss += loss.item()

# 平均损失
test_loss /= len(test_loader)
print(f'Average loss on the test dataset: {test_loss}')


Average loss on the test dataset: 3.408337164501063e-07
